
# FAISS 簡介

`faiss`（Facebook AI Similarity Search）是一個由Facebook AI Research開發的庫，用於高效地進行相似度搜索和聚類。它特別適合用於處理大型數據集，能夠快速查找高維度向量的最近鄰。

## 主要特點

- **高效的相似度搜索**：`faiss`使用優化的數據結構和算法來實現高效的最近鄰搜索，包括支持大規模數據集的多層索引。
- **多種距離度量**：支持多種距離度量，如L2距離（歐氏距離）、內積和L1距離。
- **GPU加速**：支持GPU加速，顯著提高搜索速度，特別是在處理非常大的數據集時。
- **靈活的索引結構**：提供多種索引結構，包括平面索引（Flat）、內積索引（IVF）、HNSW等，可以根據具體應用選擇合適的索引方式。

## 安裝

你可以通過`pip`安裝`faiss`：

```bash
!pip install faiss-cpu```


## 參考資料

- [FAISS GitHub Repository](https://github.com/facebookresearch/faiss)
- [FAISS 官方文件](https://faiss.ai/)


In [2]:
import numpy as np
import faiss
import time

## 範例
- 創建一個包含100,000筆128維向量的資料集。
- 創建一個包含1,000筆128維向量的查詢集，對每個查詢向量，找出在資料集中的最近的K個鄰近點。

In [3]:
# 創建一些隨機向量作為示例數據
d = 128  # 向量的維度
nb = 100000  # 資料庫中的向量數量 (number of base vectors)
nq = 1000  # 查詢向量的數量 (number of query vectors)

np.random.seed(42)  # 為了可重複性
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')


In [4]:
# 使用faiss進行搜索
def search_with_faiss(xb, xq, k):
    index = faiss.IndexFlatL2(d)  # 使用L2距離（歐氏距離）
    index.add(xb)  # 向索引中添加向量
    D, I = index.search(xq, k)  # 搜索
    return D, I

# 使用NumPy進行搜索
def search_with_numpy(xb, xq, k):
    D = np.zeros((xq.shape[0], k))
    I = np.zeros((xq.shape[0], k), dtype=int)
    for i, query in enumerate(xq):
        distances = np.linalg.norm(xb - query, axis=1)  # 計算L2距離
        indices = np.argsort(distances)[:k]  # 找到距離最近的k個向量
        D[i] = distances[indices]
        I[i] = indices
    return D, I

In [5]:
# 搜索最近鄰
k = 5  # 我們想要找到的最近鄰數量

# 計時faiss
start_time = time.time()
D_faiss, I_faiss = search_with_faiss(xb, xq, k)
faiss_time = time.time() - start_time

# 計時NumPy
start_time = time.time()
D_numpy, I_numpy = search_with_numpy(xb, xq, k)
numpy_time = time.time() - start_time


In [6]:
print(f"Faiss 搜索時間: {faiss_time:.4f} 秒")
print(f"NumPy 搜索時間: {numpy_time:.4f} 秒")

Faiss 搜索時間: 3.2680 秒
NumPy 搜索時間: 59.6542 秒


In [7]:
# 計算誤差
distance_diff = np.abs(D_faiss - D_numpy)
index_diff = np.abs(I_faiss - I_numpy)

print(f"距離誤差最大值: {np.max(distance_diff)}")
print(f"距離誤差平均值: {np.mean(distance_diff)}")
print(f"索引誤差最大值: {np.max(index_diff)}")
print(f"索引誤差平均值: {np.mean(index_diff)}")

# 驗證最近鄰索引是否一致
correct_indices = np.sum(I_faiss == I_numpy)
total_indices = I_faiss.size
print(f"最近鄰索引一致率: {correct_indices / total_indices:.4f}")

距離誤差最大值: 11.961426496505737
距離誤差平均值: 9.7812795337677
索引誤差最大值: 0
索引誤差平均值: 0.0
最近鄰索引一致率: 1.0000


In [8]:
D_numpy

array([[3.4686811 , 3.539289  , 3.54761195, 3.60324097, 3.60675883],
       [3.54771471, 3.55843329, 3.59300971, 3.60448146, 3.6630497 ],
       [3.70985723, 3.7118783 , 3.71610141, 3.747679  , 3.74931717],
       ...,
       [3.69363403, 3.74161434, 3.7549479 , 3.77117944, 3.78726339],
       [3.53720784, 3.63410711, 3.65833831, 3.67611313, 3.69432473],
       [3.7035203 , 3.74755454, 3.75149083, 3.75690532, 3.76113391]])

In [9]:
D_faiss

array([[12.031754 , 12.52655  , 12.585556 , 12.983345 , 13.0086975],
       [12.586273 , 12.662453 , 12.909721 , 12.992279 , 13.417923 ],
       [13.763031 , 13.778038 , 13.809425 , 14.045116 , 14.057388 ],
       ...,
       [13.642944 , 13.999687 , 14.099625 , 14.221764 , 14.343353 ],
       [12.511833 , 13.206711 , 13.383469 , 13.513802 , 13.648033 ],
       [13.716049 , 14.044167 , 14.073666 , 14.114334 , 14.146111 ]],
      dtype=float32)